# Isochoric Nucleation Detection analysis | stage 1: Import & Processing

As part of the PhD work of Bruno M. Guerreiro &copy; 2024.
If using this notebook, please cite the paper: https://doi.org/10.1021/acsbiomaterials.2c00075

**Disclaimer**: *due to the changing nature of programming documentation, lab work developed and tacit knowledge in this notebook, please contact the author at bruno.guerreiro@fulbrightmail.org if something is not working properly. The code is not actively maintained.*

---

### How it works: 
1. import .csv of intended data
2. change export var name at the end and write_image names
3. tinker with debugging thresholds until satisfied - mainly standard deviation
4. run all

In [1]:
### CHANGE THIS HERE ###
filename = "data/INDE_fucopol05_confirmed.csv"
molecule = 'fp05'
exportname = '' ### see end of notebook ##

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import math

# $$Data$$

### Import data here

In [3]:
### Import data

# new file (change this to assess one sample)
data = pd.read_csv(filename)

# control
#water = pd.read_csv("data/INDE_cycles/water_tony.csv")
#%store water

In [4]:
data.head()

,Time,Strain,Tset,T
0,0.000,0.000000e+00,5,12.1633
1,0.502,8.060000e-07,5,12.1458
2,1.003,1.480000e-06,5,12.1115
3,1.503,2.390000e-06,5,12.1629
4,2.004,3.650000e-06,5,12.1607


### Numerical transforms

In [5]:
### Convert time data from seconds to minutes
data['Time'] = data['Time']/60

### Convert strain data to only show x10E-4, then input on axis Strain * 10-4
data['Strain'] = np.multiply(data['Strain'], np.power(10, 4))

### Plot graph

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=data["Time"], y=data["Strain"], name="Strain", line_color="black", opacity=0.85),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=data["Time"], y=data["T"], name="T (ºC)", opacity=0.5, line_color="blue"),
    secondary_y=True,
)

# Add figure title
#fig.update_layout(title_text="INDe nucleation cycles")

# Customizable templates, just change the 'template' arg
for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
    fig.update_layout(template="simple_white", showlegend=False)

# Set x-axis title
fig.update_xaxes(title_text="<b>Time</b> (min)")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Strain</b> (V/V*1e4)", secondary_y=False)
fig.update_yaxes(title_text="<b>Temperature</b> (ºC)", secondary_y=True)

fig.show()

# $$Mining$$

### Calculate local maxima

Adapted from https://eddwardo.github.io/posts/2019-06-05-finding-local-extreams-in-pandas-time-series/

In [7]:
from scipy.signal import argrelextrema
import numpy as np

# Needed because code recognizes 'T' as Transpose
data['Temperature'] = data['T']*-1

threshold = 1

# order: How many points on each side to use for the comparison to consider
ilocs_max = argrelextrema(data.Temperature.values, np.greater_equal, order=300)[0]

# create new plot variables
Nucleation_T = data.iloc[ilocs_max].Temperature * -1
Strain_at_NT = data.iloc[ilocs_max].Strain
cycle_number = np.arange(1, len(Nucleation_T)+1)


# filter prices that are peaks and plot them differently to be visible on the plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.iloc[ilocs_max].Time, y=data.iloc[ilocs_max].Temperature, mode='lines', line_color='rgba(0,0,0,1)', line_width=1))
fig.add_trace(go.Scatter(x=data.iloc[ilocs_max].Time, y=data.iloc[ilocs_max].Temperature, text=cycle_number, marker_color='red', marker_size=5, marker_symbol=2,
                        mode='markers+text', textposition='top center', textfont=dict(color='black', size=7)
                        ))

fig.update_layout(template='simple_white', showlegend=False,
                 xaxis_title='Time (min)', yaxis_title='Nucleation temperature (ºC)')
fig.show()

### Get $T_{nucleation}$ data for each cycle

In [8]:
df = pd.DataFrame()
df.set_index = ilocs_max
df['Cycle'] = list(cycle_number)
df['Tnuc'] = list(Nucleation_T)
df['Strain'] = list(Strain_at_NT)
df

,Cycle,Tnuc,Strain
0,1,-10.6601,-1.86294
1,2,-10.6393,-1.46826
2,3,-10.8526,-1.70450
3,4,-10.6988,-1.54313
4,5,-10.6213,-1.36316
...,...,...,...
169,170,-11.2642,-0.15428
170,171,-11.1175,-0.24588
171,172,-11.1782,0.01369
172,173,-11.1336,0.00607


In [9]:
print('Number of cycles: ', df.index.stop,
     "\nAverage nucleation temperature: ", round(Nucleation_T.mean(), 2), "ºC", "+-", round(Nucleation_T.std(), 2), "ºC")

Number of cycles:  174 
Average nucleation temperature:  -10.81 ºC +- 1.88 ºC


### Remove bugged cycles with no scientific significance

In [10]:
mean = np.mean(Nucleation_T)
standard_deviation = np.std(Nucleation_T)
distance_from_mean = abs(Nucleation_T - mean)

# adjust hyperparameter as needed
max_deviations = 1

# if the datapoint deviates too much from the mean, it gets removed
not_outlier = distance_from_mean < max_deviations * standard_deviation
no_outliers = Nucleation_T[not_outlier]


print(no_outliers)
print(len(no_outliers))

1208     -10.6601
2432     -10.6393
3666     -10.8526
4898     -10.6988
6130     -10.6213
           ...   
218376   -11.2642
219645   -11.1175
220923   -11.1782
222194   -11.1336
223475   -11.2731
Name: Temperature, Length: 154, dtype: float64
154


### Adjusted number of relevant cycles

In [11]:
cycle_number_adj = np.arange(1, len(no_outliers)+1)

In [12]:
df_adj = pd.DataFrame()
df_adj['Cycle'] = list(cycle_number_adj)
df_adj['Tnuc'] = list(no_outliers)
df_adj

water = df_adj[(df_adj['Tnuc'] < -9)&(df_adj['Tnuc'] > -6)]

# Save as csv file
water.to_csv (r'data\INDe_cycles\water_bruno.csv', index = False, header=True)

In [13]:
# filter prices that are peaks and plot them differently to be visible on the plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_adj['Cycle'], y=df_adj['Tnuc'], mode='lines', line_color='rgba(0,0,0,1)', line_width=1))
fig.add_trace(go.Scatter(x=df_adj['Cycle'], y=df_adj['Tnuc'], text=cycle_number, marker_color='red', marker_size=5, marker_symbol=2,
                        mode='markers+text', textposition='top center', textfont=dict(color='black', size=8)
                        ))
fig.update_layout(template='simple_white', showlegend=False, title='Corrected cycle list',
                 xaxis_title='Time (min)', yaxis_title='Nucleation temperature (ºC)')

fig.show()

In [14]:
df_adj['Tnuc'].describe()

count    154.000000
mean     -10.862687
std        0.401648
min      -11.751500
25%      -11.067725
50%      -10.897700
75%      -10.707550
max       -8.938100
Name: Tnuc, dtype: float64

### Nucleation temperature distribution

In [15]:
import plotly.graph_objects as go

import pandas as pd

fig = go.Figure(data=go.Violin(y=df_adj['Tnuc'], box_visible=True, line_color='black',
                               meanline_visible=True, fillcolor='lightblue', opacity=0.6, points="all",
                               x0='Water', pointpos=0))

# Customizable templates, just change the 'template' arg
for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
    fig.update_layout(template="simple_white")

fig.update_layout(yaxis_zeroline=False, width = 600,height = 600, yaxis_title='Nucleation temperature (ºC)')
fig.show()

### Nucleation cycles, survival graph, violin/box plots

In [16]:
### SURVIVAL GRAPH CALCULATIONS

# Obtain y-axis: Order nucleation temperatures from smallest to largest
ordered_Tnuc = sorted(df_adj['Tnuc'], reverse=False)

# Obtain x-axis: Divide index by length of array
chi = []
for i in range(0,len(ordered_Tnuc)):
    chi.append((i+1)/len(ordered_Tnuc))

$$\chi(T) = e^{\frac{-A}{\beta}\frac{\gamma(T-T_m)^{1+n}}{1+n}}$$

In [17]:
fig = make_subplots(rows=1, cols=3)

# Tnuc-cycle plot
fig.add_trace(go.Scatter(x=df_adj['Cycle'], y=df_adj['Tnuc'], mode='lines+markers', legendgroup=1), row=1,col=1)

# Survivor plot
fig.add_trace(go.Scatter(x=chi, y=ordered_Tnuc, mode='markers', marker_size=8, opacity=0.75, legendgroup=2), row=1,col=2)
#POISSON FIT HERE --- fig.add_trace(go.Scatter(x=chi, y=ordered_Tnuc, mode='lines', line_color='black', line_width=2, opacity=.35), row=1,col=3)

# Violin plot
fig.add_trace(go.Violin(y=water['Tnuc'], name=molecule, box_visible=True, points='all', meanline_visible=True, pointpos=0, marker_opacity=.35, opacity=.75, marker_size=5, legendgroup=3), row=1, col=3)
fig.add_trace(go.Violin(y=df_adj['Tnuc'], name=molecule, box_visible=True, points='all', meanline_visible=True, pointpos=0, marker_opacity=.35, marker_size=5, legendgroup=3), row=1, col=3)

# Box plot
#fig.add_trace(go.Box(y=water['Tnuc'], boxpoints='all', pointpos=0, marker_color='rgb(107,174,214)', marker_opacity=.35, marker_size=8, line_color='rgb(107,174,214)', name="without", legendgroup=4), row=1, col=4)
#fig.add_trace(go.Box(y=sample['Tnuc'], boxpoints='all', pointpos=0, marker_opacity=.35, marker_size=8, name="with", legendgroup=4), row=1, col=4)

fig.update_layout(template='simple_white', 
                  showlegend=False,
                  xaxis1_title = 'Cycles',
                  xaxis2_title = 'Unfrozen fraction (%)',
                  xaxis3_title = 'Concentration (wt%)',
                  #xaxis4_title = 'water',
                  yaxis1_title = 'Nucleation temperature (ºC)')

fig.show()

In [18]:
### EXPORTS FOR NEXT STAGE (change 'sample' for desired var name) ###

fp05 = df_adj
%store fp05

Stored 'fp05' (DataFrame)


## --> GO TO STAGE 2